# Importing libraries 

In [ ]:
import os
import shutil
import nibabel as nib
import numpy as np
from glob import glob
import imageio
from termcolor import colored
import registration as reg 
import skull_stripping as ss
import enhancement as enh

# Initials

In [ ]:
def make_dirs(path_lst):
    for path in path_lst:
        if not os.path.exists(path):
            os.makedirs(path)
        else:
            pass

# make outputdir and return output path (including filename) by combining outDir, pt, filename
def get_outpath(outDir, pt, filename):
    outPath = os.path.join(outDir, pt, filename)

    make_dirs([os.path.join(outDir, pt)])

    if os.path.exists(outPath):
        print('{} already exists'.format(filename))
        return None
    else:
        return outPath
    

sourceDir = '/media/sf_sharedfolder/Converted'
targetDir = '/media/sf_sharedfolder/Output'
maskDir = None
outputPath = '/media/sf_sharedfolder/MONSTR/Output'

resampleDir = os.path.join(targetDir, "01_Resampled")
correctionDir = os.path.join(targetDir, "02_Corrected")
registerDir = os.path.join(targetDir, '03_Registered_corrected')
strippedDir = os.path.join(targetDir, '04_Stripped_wo_Normalized')
enhDir = os.path.join(targetDir, '05_Histogram_Enhancement')
znormedDir = os.path.join(targetDir, '06_Stripped_znormalized')
sliceDir = os.path.join(targetDir, '07_Slice_Extraction')

make_dirs([targetDir, resampleDir, correctionDir, registerDir, enhDir, strippedDir, znormedDir, sliceDir])


# Resampling 

In [ ]:
total = 0

for i, sub in enumerate(os.listdir(sourceDir)):
    os.chdir(os.path.join(sourceDir, sub))
    for j, scan in enumerate(os.listdir()):
        total += 1
        path = os.path.dirname(scan)
        file = os.path.basename(scan)
        print("Resampling: #{}, {}".format(total, file))
        pt = path.split(os.path.sep)[-1]
        
        os.mkdir(os.path.join('/media/sf_sharedfolder/Output/01_Resampled', sub))
        resample_outPath = get_outpath(os.path.join(resampleDir, pt, sub), "", file)
     
        if resample_outPath is None:
            continue
        
        os.system(ResampleImage, 3, {scan}, {resample_outPath} ,1x1x1)
        os.chdir(sourceDir)

print(colored('########## Resampling Completed ##########', 'white', 'on_green'))  


#  N4 Bias Correction

In [ ]:
total = 0

for i, sub in enumerate(os.listdir(resampleDir)):
    os.chdir(os.path.join(resampleDir, sub))
    for j, scan in enumerate(os.listdir()):
        total += 1
        path = os.path.dirname(scan)
        file = os.path.basename(scan)
        print("N4 Bias Correction: #{}, {}".format(total, file))
        pt = path.split(os.path.sep)[-1]
        
        os.mkdir(os.path.join('/media/sf_sharedfolder/Output/02_Corrected', sub))
        corr_outPath = get_outpath(os.path.join(correctionDir, pt, sub), "", file)
     
        if corr_outPath is None:
            continue
        
        !N4BiasFieldCorrection -d 3 -i {scan} -o {corr_outPath} -s 3 -c [50x50x50x50,0.00001] -b [150]
        os.chdir(resampleDir)

print(colored('########## N4 Bias Correction Completed ##########', 'white', 'on_green'))      


# Registration

In [ ]:
template = '/media/sf_sharedfolder/Code/MNI152_T1_1mm.nii.gz'
total = 0

for i, sub in enumerate(os.listdir(correctionDir)):
    os.chdir(os.path.join(correctionDir, sub))
    for j, scan in enumerate(os.listdir()):
        total += 1
        path = os.path.dirname(scan)
        file = os.path.basename(scan)
        print("Registering: #{}, {}".format(total, file))
        pt = path.split(os.path.sep)[-1]
        
        os.mkdir(os.path.join('/media/sf_sharedfolder/Output/03_Registered_corrected', sub))
        reg_path = get_outpath(os.path.join(registerDir, pt, sub), "", file)
     
        if reg_path is None:
            continue
        
        reg.registration(scan, reg_path, template)
        
        
        os.chdir(correctionDir)

print(colored('########## Registration Completed ##########', 'white', 'on_green'))      


# Skull Stripping 

In [ ]:
total = 0

for i, sub in enumerate(os.listdir(registerDir)):
    os.chdir(os.path.join(registerDir, sub))
    for j, scan in enumerate(os.listdir()):
        total += 1
        path = os.path.dirname(scan)
        file = os.path.basename(scan)
        print("Stripping: #{}, {}".format(total, file))
        pt = path.split(os.path.sep)[-1]
        
        os.mkdir(os.path.join('/media/sf_sharedfolder/Output/04_Stripped_wo_Normalized', sub))
        ss_path = get_outpath(os.path.join(strippedDir, pt, sub), "", file)
     
        if ss_path is None:
            continue
        
        ss.strip_skull(scan, ss_path)
        
        os.chdir(registerDir)

print(colored('########## Skull Stripping Completed ##########', 'white', 'on_green'))          


# ZScore Normalization

In [ ]:
total = 0

for i, sub in enumerate(os.listdir(strippedDir)):
    os.chdir(os.path.join(strippedDir, sub))
    for j, scan in enumerate(os.listdir()):
        total += 1
        path = os.path.dirname(scan)
        file = os.path.basename(scan)
        print("Slice Extraction: #{}, {}".format(total, file))
        pt = path.split(os.path.sep)[-1]
        
        os.mkdir(os.path.join('/media/sf_sharedfolder/Output/06_Stripped_znormalized', sub))
        z_path = get_outpath(os.path.join(znormedDir, pt, sub), "", file)
     
        if z_path is None:
            continue
        
        img_nii = nib.load(os.path.join(path, scan))
        img_data = img_nii.get_data()
        mean, std = np.mean(img_data), np.std(img_data)
        normalized = nib.Nifti1Image((img_data - mean) / std, img_nii.affine, img_nii.header)
        nib.save(normalized, z_path)
        
        os.chdir(strippedDir)

print(colored('########## ZScore Normalization Completed ##########', 'white', 'on_green'))  


# Slice Extraction

In [ ]:
total = 0

for i, sub in enumerate(os.listdir(znormedDir)):
    os.chdir(os.path.join(znormedDir, sub))
    for j, scan in enumerate(os.listdir()):
        total += 1
        path = os.path.dirname(scan)
        file = os.path.basename(scan)
        print("Slice Extraction: #{}, {}".format(total, file))
        pt = path.split(os.path.sep)[-1]
        
        os.mkdir(os.path.join('/media/sf_sharedfolder/Output/07_Slice_Extraction', sub))
        slice_path = get_outpath(os.path.join(sliceDir, pt, sub), "", file)
     
        if slice_path is None:
            continue
        
        img_nii = nib.load(os.path.join(path, scan))
        img_data = img_nii.get_fdata()
        
        (x, y, z) = img_data.shape
        mid = x//2
        
        for k in range(mid-7, mid+7):
            slice = img_data[:, :, k]
            
            imageio.imwrite(os.path.join(sliceDir, sub, '{}.png'.format(sub + "_" + str(k))), slice)
            
        
        os.chdir(znormedDir)

print(colored('########## Slice Extraction Completed ##########', 'white', 'on_green'))  
